# Frame Sequence Extractor

Extracts frame sequences from the dataset.

### Preparation

- Place the h2odataset in the following directory relative to the workspace:
`data/h2o/h2odataset`

### Run

- Import the required packages and set the required parameters
- Select the required mode by running the corresponding cell (only)
- If you use a pre-computed sample_ids array, make sure to have placed it in `data/h2o/seq_n_mode/sample_ids.npy`
- Follow along the cells in a chapter to extract the corresponding data

### Output

- The extracted data is saved in numpy's .npy files in `data/h2o` in corresponding folders. 
- For example, the 8 frame sequences for training are stored int `data/h2o/frame_seq_8_train/`.
- Images are stored in the format HxWxC in RGB format
- The name of the files corresponds to the id provided in the action files of the dataset (e.g. 001.npy).

---

#### Imports

In [2]:
import os
import cv2
import numpy as np
from numpy.typing import NDArray
import pandas as pd
from tqdm import tqdm
from typing import List

#### Parameters

In [4]:
n_frames_per_seq : int = 8
h2o_root = '../data/h2o/'
h2odataset_root = '../data/h2o/h2odataset/'
h2o_actions = h2odataset_root + 'action_labels/'

## Choose Mode

> (Only) Run the cell corresponding to the requested mode (train, val).

The consequent extractions automatically extract data of the configured mode.

Training Set

In [5]:
mode = 'train'
sample_root = h2o_root + f'seq_{n_frames_per_seq}_{mode}/'

df = pd.read_csv(h2o_actions + 'action_train.txt', delimiter=' ')
ids : List[int] = df['id'].to_list()
paths : List[str] = df['path'].to_list()
labels : List[int] = df['action_label'].to_list()
start_acts : List[int] = df['start_act'].to_list()
end_acts : List[int] = df['end_act'].to_list()

print("Number of action sequences:", len(ids))

Number of action sequences: 569


Validation Set

In [17]:
mode = 'val'
sample_root = h2o_root + f'seq_{n_frames_per_seq}_{mode}/'

df = pd.read_csv(h2o_actions + 'action_val.txt', delimiter=' ')
ids : List[int] = df['id'].to_list()
paths : List[str] = df['path'].to_list()
labels : List[int] = df['action_label'].to_list()
start_acts : List[int] = df['start_act'].to_list()
end_acts : List[int] = df['end_act'].to_list()

print("Number of action sequences:", len(ids))

Number of action sequences: 122


## Compute Sample IDs

Sample the frame ids from the action sequences.

In [19]:
os.makedirs(sample_root, exist_ok=True) # create the destination directory

sample_ids : List[NDArray] = []
# sample the frames for each action
for i in tqdm(range(len(ids))):
    start_act = start_acts[i]
    end_act = end_acts[i]
    n_frames_total = end_act-start_act+1 # total number of frames in the action sequence
    assert n_frames_total >= n_frames_per_seq, \
        f"Requested {n_frames_per_seq} samples, but action (id {ids[i]}) has only {n_frames_total} frames"

    seq_ids = np.linspace(start_acts[i], end_acts[i], n_frames_per_seq, dtype=int)
    sample_ids.append(seq_ids)

sample_ids = np.array(sample_ids)
np.save(sample_root + 'sample_ids.npy', sample_ids)
print("Sampling IDs shape:", sample_ids.shape) # sanity check the shape of the sampled ids
print("Last sampled sequence ids:", seq_ids) # sanity check the sampling from the last action

100%|██████████| 122/122 [00:00<00:00, 21045.70it/s]

Sampling IDs shape: (122, 8)
Last sampled sequence ids: [476 485 495 504 514 523 533 543]


## Action Labels

In [20]:
action_labels_dest = h2o_root + 'action_labels_' + mode + '.npy'
action_labels = np.array(labels)
np.save(action_labels_dest, action_labels)

## Frame Sequences

Helper Functions

In [21]:
def check_img_sample(img_path: str):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    print("Img shape:", img.shape)
    print("Img dtype:", img.dtype)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [22]:
def sample_frame_sequence(act_path: str, sample_ids: NDArray) -> NDArray:
    """
    Sample frames from an action sequence in the dataset.

    Args:
    -----
        act_path (str): path to the action sequence images
        sample_ids (NDArray): array of frame ids to sample

    Returns:
    --------
        NDArray: array of the sampled frame sequence
    """

    frames : list[NDArray] = []
    for id in sample_ids:
        img_path = act_path + f'{id:06d}.png'
        img : NDArray = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        frames.append(img)
    
    return np.array(frames)

#### Prepare Extraction

Check whether the images are extracted correctly.

In [6]:
# create the destination directories
sample_frames_dest = sample_root + f'frames_{mode}/'
os.makedirs(sample_frames_dest, exist_ok=True) # create the destination directory

# load the sample ids from file
sample_ids = np.load(sample_root + 'sample_ids.npy')
print("Sample IDs shape:", sample_ids.shape)

# check whether images are loaded correctly
check_img_sample(h2odataset_root + paths[0] + '/cam4/rgb/000000.png') # TODO check path

Sample IDs shape: (569, 8)


NameError: name 'check_img_sample' is not defined

#### Extraction

In [24]:
for i in tqdm(range(len(ids))):
    # TODO check path to image data
    frames = sample_frame_sequence(h2odataset_root + paths[i] + '/cam4/rgb/', sample_ids[i])
    np.save(sample_frames_dest + f'{ids[i]:03d}.npy', frames)
print("Images sampled and saved successfully!")

100%|██████████| 122/122 [01:12<00:00,  1.68it/s]

Images sampled and saved successfully!


## Pose Sequences

Helper Function

In [7]:
def sample_pose_sequence(pose_path: str, sample_ids: NDArray) -> NDArray:
    """
    Sample poses from an action sequence in the dataset.

    Args:
    -----
        pose_path (str): path to the action sequence images
        sample_ids (NDArray): array of frame ids to sample

    Returns:
    --------
        NDArray: array of the sampled frame sequence
    """

    poses : list[NDArray] = []
    for id in sample_ids:
        pose_file = pose_path + f'{id:06d}.txt'
        pose = np.loadtxt(pose_file, dtype=np.double)

        poses.append(pose)
    
    return np.array(poses)

In [8]:
def transform_to_rt(rt_vecs: NDArray):
    """
    Transform the raw object_rt vectors to 4x4 transformation matrices.

    Args:
    -----
        rt_vecs (NDArray): Nx17 array of raw object_rt vectors

    Returns:
    --------
        NDArray: Nx4x4 array of transformation matrices
    """

    # remove the first number from each vector
    rt_vecs = rt_vecs[:, 1:]
    # reshape the vector
    rt_vecs = rt_vecs.reshape(-1, 4, 4)
    
    return rt_vecs
    

#### Prepare Extraction

Check whether poses are extracted correctly.

In [9]:
# create the destination directories
sample_hand_poses_dest = sample_root + f'poses_hand_{mode}/'
sample_obj_poses_dest = sample_root + f'poses_obj_{mode}/'
sample_obj_rt_dest = sample_root + f'obj_rt_{mode}/'
os.makedirs(sample_hand_poses_dest, exist_ok=True) # create the destination directory
os.makedirs(sample_obj_poses_dest, exist_ok=True) # create the destination directory
os.makedirs(sample_obj_rt_dest, exist_ok=True) # create the destination directory

# load the sample ids from file
sample_ids = np.load(sample_root + 'sample_ids.npy')
print("Sample IDs shape:", sample_ids.shape)

# check whether poses are loaded correctly
poses = sample_pose_sequence(h2odataset_root + 'data_pose/' + paths[0] + '/cam4/hand_pose/', sample_ids[0])
print("Hand pose shape:", poses.shape)
print(poses[0, -1]) # sanity check
poses = sample_pose_sequence(h2odataset_root + 'data_pose/' + paths[0] + '/cam4/obj_pose/', sample_ids[0])
print("Object pose shape:", poses.shape)
print(poses[0, -1]) # sanity check
poses = sample_pose_sequence(h2odataset_root + 'data_pose/' + paths[0] + '/cam4/obj_pose_rt/', sample_ids[0])
print("Object pose RT shape:", poses.shape)
print(poses[0, -1]) # sanity check


Sample IDs shape: (569, 8)
Hand pose shape: (8, 128)
0.6218021202
Object pose shape: (8, 64)
0.5920540018426089
Object pose RT shape: (8, 17)
1.0


#### Extraction

Hand and Object Poses

In [9]:
for i in tqdm(range(len(ids))):
    poses = sample_pose_sequence(h2odataset_root + 'data_pose/' + paths[i] + '/cam4/hand_pose/', sample_ids[i])
    np.save(sample_hand_poses_dest + f'{ids[i]:03d}.npy', poses)
    poses = sample_pose_sequence(h2odataset_root + 'data_pose/' + paths[i] + '/cam4/obj_pose/', sample_ids[i])
    np.save(sample_obj_poses_dest + f'{ids[i]:03d}.npy', poses)
print("Poses sampled and saved successfully!")

100%|██████████| 569/569 [00:03<00:00, 159.52it/s]

Poses sampled and saved successfully!


Object Pose RT Matrices

In [10]:
for i in tqdm(range(len(ids))):
    poses = sample_pose_sequence(h2odataset_root + 'data_pose/' + paths[i] + '/cam4/obj_pose_rt/', sample_ids[i])
    poses = transform_to_rt(poses)
    np.save(sample_obj_rt_dest + f'{ids[i]:03d}.npy', poses)
print("Object RT sampled and saved successfully!")

100%|██████████| 569/569 [00:03<00:00, 170.38it/s]

Object RT sampled and saved successfully!
[[[-0.35512614  0.93421651 -0.03353196 -0.07540931]
  [-0.82986139 -0.29853773  0.47138616 -0.02668573]
  [ 0.43036657  0.19522853  0.88128898  0.46797309]
  [ 0.          0.          0.          1.        ]]

 [[-0.35210428  0.93345575 -0.06842806 -0.06834233]
  [-0.85278704 -0.28982704  0.43445847 -0.02316135]
  [ 0.38571584  0.21132935  0.89808853  0.46651409]
  [ 0.          0.          0.          1.        ]]

 [[-0.30426001  0.94697136 -0.10329834 -0.05765282]
  [-0.88604224 -0.24151667  0.39572523 -0.01969273]
  [ 0.34979256  0.21193016  0.91254074  0.46393074]
  [ 0.          0.          0.          1.        ]]

 [[-0.23206954  0.96220823 -0.14247282 -0.04908071]
  [-0.93334112 -0.17903407  0.31116036 -0.00725253]
  [ 0.27389392  0.20518668  0.93961714  0.4582273 ]
  [ 0.          0.          0.          1.        ]]

 [[-0.11534512  0.98658798 -0.11549512 -0.0380447 ]
  [-0.95464874 -0.0779731   0.28734222  0.0020412 ]
  [ 0.27448324